# Line chart (benchmark analysis)

In [ ]:

import pandas as pd
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro
from vizro.tables import dash_ag_grid
from vizro.actions import export_data, filter_interaction
import plotly.graph_objects as go
from vizro.models.types import capture

Vizro._reset()

gapminder = pd.read_csv('uni.csv', header=0, usecols=[
    "University Name", 
    "Academic reputation (QS25)", 
    "Teaching quality (THE24)", 
    "Employment Outcomes (QS25)", 
    "Research quality (THE24)", 
    "International Research Network (QS25)", 
    "Employer Reputation (QS25)"
])

gapminder_radar=pd.DataFrame(dict(
    r=[82.2, 39.7, 92.9, 88.3, 87.4, 44.7],
    theta=['Academic reputation (QS25)', 'Teaching quality (THE24)', 'Employment Outcomes (QS25)', 'Research quality (THE24)', 'International Research Network (QS25)', 'Employer Reputation (QS25)']))


# Calculate the national rankings for each year


#Process data

@capture("graph")

def radarchart(data_frame, r, theta, title=None, markers=None, hover_name=None, line_close=None): 
    fig_radar = px.line_polar(data_frame=data_frame, r=r, theta=theta, title=title, markers=markers, hover_name=hover_name, line_close=line_close)
    
    fig_radar.update_layout(
        title=go.layout.Title(
            text="University of Auckland",
            font=dict(
                family="Tahoma, sans-serif, bold",
                size=25,)

            ),
    
            legend=dict(
                font=dict(
                    family="Tahoma, sans-serif",
                    size=12,
        )
    ),
        
   annotations=[
            dict(
                xref='paper', 
                yref='paper',
                x=0.5, 
                y=-0.15,  # Position below the chart
                showarrow=False,
                text="Source: <a href='https://www.topuniversities.com/universities/university-auckland'>QS Rankings</a>, <a href='https://www.timeshighereducation.com/world-university-rankings'>THE Rankings</a>",
                font=dict(
                    family="Tahoma, sans-serif",
                    size=12,
                ),
            )
        ],
    ) 
    
    fig_radar.update_traces(fill='toself')
    
    return fig_radar

def create_metrics():
    """Function returns a page to show scores of different metrics on each university."""

   
    # Benchmark analysis
    page_years = vm.Page(
        title="Universities' Scores in metrics",
        description="Discovering how different NZ universities are scored with different metrics",
        layout=vm.Layout(grid=[[0]]),
        components=[
            vm.Graph(
                id="radar_university",
                figure=radarchart(
                    gapminder_radar,
                    r="r",
                    theta="theta",
                    markers=True,
                    hover_name="theta",
                    line_close=True
                ),
            ),
        ],
        # Uncomment the following lines if needed
        # controls=[
        #     vm.Filter(column="University Name", selector=vm.Dropdown(value="ALL", multi=True, title="Select University")),
        #     # vm.Filter(column="Year", selector=vm.RangeSlider(title="Select Year", step=1, marks=None)),
        #     vm.Parameter(
        #         targets=["radar_university.y"],
        #         selector=vm.Dropdown(
        #             options=["International Rankings", "National Rankings"], multi=False, value="International Rankings", title="Choose between International and National Rankings"
        #         ),
        #     ),
            
        # ],
    )
    return page_years

# # Example usage
dashboard = vm.Dashboard(
    pages=[
        create_metrics()
    ],
    navigation=vm.Navigation(
        nav_selector=vm.NavBar(
            items=[
                vm.NavLink(label="Universities' Scores in metrics", pages=["Universities' Scores in metrics"], icon="Stacked Bar Chart"),
            ]
        ),
    ),
)

IS_JUPYTERLAB = 'true'

if not IS_JUPYTERLAB:
    app = Vizro().build(dashboard)
    server = app.dash.server
    
    if __name__ == "__main__":  
        app.run(port=8080)
else:
    Vizro(assets_folder="assets").build(dashboard).run(port=8083)

